# **저시력자를 위한 원화 화폐 분류**

## 라이브러리 불러오기

In [1]:
import glob, shutil, os, splitfolders
import json
import yaml

## 전처리

### 데이터 이동

In [2]:
money_list = ["10", "50", "100", "500", "1000", "5000", "10000", "50000"]
data_path = "./money_dataset/"

In [5]:
# image to dst
for money in money_list:
    src_dir = os.path.join("./money_dataset/", money)
    dst_dir = os.path.join("./money_dataset/images", money)
    os.makedirs(dst_dir, exist_ok=True)
    images_file_list = [filename for filename in os.listdir(src_dir) if filename.endswith(".jpg")]
    for filename in images_file_list:
        src_path = os.path.join(src_dir, filename)
        dst_path = os.path.join(dst_dir, filename)
        shutil.move(src_path, dst_path)

In [7]:
# label to dst
for money in money_list:
    src_dir = os.path.join("./money_dataset/", money)
    dst_dir = os.path.join("./money_dataset/labels", money)
    os.makedirs(dst_dir, exist_ok=True)
    label_file_list = [filename for filename in os.listdir(src_dir) if filename.endswith(".json")]
    for filename in label_file_list:
        src_path = os.path.join(src_dir, filename)
        dst_path = os.path.join(dst_dir, filename)
        shutil.move(src_path, dst_path)

In [74]:
# images train val split
images_path = "./money_dataset/images"
splitfolders.ratio(images_path, output="./money_dataset/images", seed=123, ratio=(.7, .3), group_prefix=None)

In [41]:
# 균등 배분 비율 확인
for i in money_list:
    print(f'{i}원 train 갯수:',len(os.listdir('./money_dataset/images/train/' + i)))
for i in money_list:
    print(f'{i}원 val 갯수:',len(os.listdir('./money_dataset/images/val/' + i)))
train_sum = 0
val_sum = 0
total_sum = 0
for i in money_list:
    train_sum += len(os.listdir('./money_dataset/images/train/' + i))
for i in money_list:
    val_sum +=len(os.listdir('./money_dataset/images/val/' + i))
print('총 train 갯수:',train_sum)
print('총 val 갯수:',val_sum)
print('총 data 갯수:', train_sum + val_sum)

10원 train 갯수: 305
50원 train 갯수: 308
100원 train 갯수: 308
500원 train 갯수: 308
1000원 train 갯수: 600
5000원 train 갯수: 606
10000원 train 갯수: 606
50000원 train 갯수: 609
10원 val 갯수: 131
50원 val 갯수: 132
100원 val 갯수: 132
500원 val 갯수: 132
1000원 val 갯수: 258
5000원 val 갯수: 261
10000원 val 갯수: 261
50000원 val 갯수: 261
총 train 갯수: 3650
총 val 갯수: 1568
총 data 갯수: 5218


In [16]:
## 단위 구분 없이 train, val 각 폴더에 모든 권종 모아놓기
train_dst = "./money_dataset/yolov5_dataset/images/trainset"
val_dst = "./money_dataset/yolov5_dataset/images/validset"

os.makedirs(train_dst, exist_ok=True)
os.makedirs(val_dst, exist_ok=True)

# 단위 구분 없이 train 각 폴더의 이미지를 trainset 폴더로 복사하기
for money in money_list:
    train_src = os.path.join("./money_dataset/images/train/", money)
    for filename in os.listdir(train_src):
        src_path = os.path.join(train_src, filename)
        dst_path = os.path.join(train_dst, filename)
        shutil.copy2(src_path, dst_path)

# 단위 구분 없이 val 각 폴더의 이미지를 validset 폴더로 복사하기
for money in money_list:
    val_src = os.path.join("./money_dataset/images/val/", money)
    for filename in os.listdir(val_src):
        src_path = os.path.join(val_src, filename)
        dst_path = os.path.join(val_dst, filename)
        shutil.copy2(src_path, dst_path)

In [76]:
# 라벨 데이터도 단위 구분 없이 train, val 각 폴더로
train_labels_dst = "./money_dataset/labels/trainset"
val_labels_dst = "./money_dataset/labels/validset"

os.makedirs(train_labels_dst, exist_ok=True)
os.makedirs(val_labels_dst, exist_ok=True)

# 이미지 파일명을 바탕으로 라벨 파일을 trainset 폴더로 복사하기
for file_name in os.listdir(train_dst):
    label_file_name = file_name.replace(".jpg", ".json")
    
    for money in money_list:
        src_path = os.path.join(f"./money_dataset/labels/{money}/", label_file_name)
        if os.path.exists(src_path):
            dst_path = os.path.join(train_labels_dst, label_file_name)
            shutil.copy2(src_path, dst_path)
            break

# 이미지 파일명을 바탕으로 라벨 파일을 validset 폴더로 복사하기
for file_name in os.listdir(val_dst):
    label_file_name = file_name.replace(".jpg", ".json")

    for money in money_list:
        src_path = os.path.join(f"./money_dataset/labels/{money}/", label_file_name)
        if os.path.exists(src_path):
            dst_path = os.path.join(val_labels_dst, label_file_name)
            shutil.copy2(src_path, dst_path)
            break

In [50]:
train_images_list = os.listdir(train_dst)
train_labels_list = os.listdir(train_labels_dst)
val_images_list = os.listdir(val_dst)
val_labels_list = os.listdir(val_labels_dst)

In [51]:
print(len(train_images_list))
print(len(train_labels_list))
print(len(val_images_list))
print(len(val_labels_list))

3650
3650
1568
1568


------------------------------------------

### json 파일 정보 추출

In [5]:
# 라벨 정보 변경 사전
class_dict = {'Ten' : 0,
              'Fifty' : 1,
              'Hundred' : 2,
              'Five_Hundred' : 3,
              'Thousand' : 4,
              'Five_Thousand' : 5 ,
              'Ten_Thousand' : 6,
              'Fifty_Thousand' : 7}

In [17]:
def process_json(json_path, txt_path):
    with open(json_path, "r") as f:
        data = json.load(f)
    
    annotations = data["shapes"]

    # 이미지 원본대비 1/5
    image_width = data["imageWidth"] / 5
    image_height = data["imageHeight"] / 5

    yolo_labels = []
    for anno in annotations:
        label = anno["label"].replace("_front", "").replace("_back", "") # 앞뒤 구분 제거
        label_index = class_dict[label] # 클래스 인덱스 사용
        bbox = anno["points"]

        x1, y1 = bbox[0]
        x2, y2 = bbox[1]

        #  이미지 원본대비 1/5
        x1, y1, x2, y2 = x1 / 5, y1 / 5, x2 / 5, y2 / 5

        x_center = (x1 + x2) / 2 / image_width
        y_center = (y1 + y2) / 2 / image_height
        width_norm = abs(x2 - x1) / image_width
        height_norm = abs(y2 - y1) / image_height

        yolo_label = f"{label_index} {x_center} {y_center} {width_norm} {height_norm}\n"
        yolo_labels.append(yolo_label) 

    with open(txt_path, "w") as f:
        f.writelines(yolo_labels)

In [18]:
# 경로 지정
train_json_dir = "./money_dataset/labels/trainset/"
train_txt_dir = "./money_dataset/yolov5_dataset/labels/trainset"
os.makedirs(train_txt_dir, exist_ok=True)

val_json_dir = "./money_dataset/labels/validset/"
val_txt_dir = "./money_dataset/yolov5_dataset/labels/validset"
os.makedirs(val_txt_dir, exist_ok=True)

In [19]:
# 데이터 추출
for json_file in os.listdir(train_json_dir):
    json_path = os.path.join(train_json_dir, json_file)
    txt_path = os.path.join(train_txt_dir, json_file.replace(".json", ".txt"))

    if json_file.endswith(".json"):
        process_json(json_path, txt_path)

for json_file in os.listdir(val_json_dir):
    json_path = os.path.join(val_json_dir, json_file)
    txt_path = os.path.join(val_txt_dir, json_file.replace(".json", ".txt"))

    if json_file.endswith(".json"):
        process_json(json_path, txt_path)

### Dataset yaml파일 생성

In [9]:
money_dict = {0: '10', 1: '50', 2: '100', 3: '500', 4: '1000', 5: '5000', 6: '10000', 7: '50000'}

# 지정
money_yaml = {
  'path': '/home/hwl/Workspace/Money_Object_Detection/money_dataset/',
  'train': 'yolov5_dataset/images/trainset',
  'val': 'yolov5_dataset/images/validset',
  'names': money_dict
}

# 저장
with open('./money_dataset/money.yaml', 'w') as f:
    yaml.dump(money_yaml, f)

## 모델링

In [21]:
# 경로 지정
train_py = "/home/hwl/Workspace/yolov5/train.py"
data = "/home/hwl/Workspace/Money_Object_Detection/money_dataset/money.yaml"
cfg = "/home/hwl/Workspace/yolov5/models/money_detection_yolov5s.yaml"
weights = "/home/hwl/Workspace/yolov5/weights/yolov5s.pt"

In [ ]:
# 훈련
!python {train_py} --img 640 --batch 16 --epochs 10 --data {data} --cfg {cfg} --weights {weights}